> First time use: follow instructions in the README.md file in this directory.


**[PT]** Português

---

**[EN]** English


# Naturalidade


Análise dos valores da informação sobre a naturalidade dos estudantes

---

# Place of Birth

Information related to place of birth of students


## Setup

In [30]:
from timelinknb import get_db
from ucalumni.config import default_db

db_spec = default_db
db = get_db(db_spec)

## Lista de lugares diferentes e número de ocorrências

---

## List of different places with number of occurrences

In [175]:
from timelinknb.pandas import attribute_values

naturalidades = attribute_values('naturalidade',dates_between=('1500-00-00','1990-00-00'))
naturalidades.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11499 entries, Lisboa to Óvoa, Viseu
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   count     11499 non-null  int64 
 1   date_in   11499 non-null  object
 2   date_max  11499 non-null  object
dtypes: int64(1), object(2)
memory usage: 359.3+ KB


### Lugares principais

---

### Main locations

In [32]:
naturalidades.sort_values('count', ascending=False).head(20)



,count,date_in,date_max
value,,,
Lisboa,8784,1537-02-12,1916-07-19
Coimbra,5526,1537-00-00,1915-10-12
Porto,3391,1537-05-30,1917-10-22
Braga,1608,1540-01-21,1914-07-24
Évora,1072,1537-11-22,1910-10-10
Viseu,986,1537-00-00,1912-07-03
Guimarães,980,1537-12-18,1912-07-18
Lamego,972,1537-00-00,1909-10-05
Aveiro,790,1538-04-21,1913-10-13


### Lugares com menos de três estudantes

---

### Locations with less than three students

In [33]:
naturalidades[naturalidades['count']<3].info()

<class 'pandas.core.frame.DataFrame'>
Index: 8948 entries, Abadia to Óvoa, Viseu
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   count     8948 non-null   int64 
 1   date_in   8948 non-null   object
 2   date_max  8948 non-null   object
dtypes: int64(1), object(2)
memory usage: 279.6+ KB


In [34]:
naturalidades[naturalidades['count']<3].sort_values('count', ascending=False).head()

,count,date_in,date_max
value,,,
Abadia,2,1836-10-14,1886-10-14
"Sande, Viseu",2,1840-10-23,1855-10-13
Santa Comba da Ermida,2,1766-10-01,1824-10-08
"Santa Clara, Coimbra",2,1589-10-14,1910-10-12
"Santa Catarina, Alcobaça",2,1613-10-09,1760-10-01


## Identificação de topónimos

---

## Geocoding

https://craftingdh.netlify.app/tutorials/folium/

In [35]:
!pip install geopy

This is here for testing purposes will migrate to timelinknb when done.

In [39]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.distance import geodesic
import pandas as pd

class GeocodeOSM:

    result_cols = ['place_name',
                    'geocoder',
                    'id',
                    'address',
                    'city',
                    'country',
                    'importance',
                    'class',
                    'type',
                    'latitude',
                    'longitude',
                    'distance']

    def __init__(self,user_agent, timeout=3, min_delay_seconds=1, error_wait_seconds=10):
        """ Create a geocoder for Open Street Mapp (Nominatim)
        
        Args:
            user_agent: string identifying the application.
            timeout: time in seconds to wait for answer
            min_delay_seconds: delay between calls to service
            error_wait_seconds: seconds to wait when service return error

        Creates a utility class around geopy Nominating geocoder.

        Usage:
            osm_coder = GeocodeOSM('myapp')
            result = osm_coder.geocode_osm('Coimbra')

        See:
            https://geopy.readthedocs.io/en/stable/#nominatim
            https://geopy.readthedocs.io/en/stable/index.html?highlight=geopy.extra.rate_limiter.RateLimiter#geopy.extra.rate_limiter.RateLimiter
        """
        self.geolocator = Nominatim(user_agent=user_agent, timeout=timeout)
        # abide to https://operations.osmfoundation.org/policies/nominatim/
        self.geocode = RateLimiter(geolocator.geocode, min_delay_seconds = min_delay_seconds,  max_retries=5,
                                    error_wait_seconds=error_wait_seconds, return_value_on_exception = None) 

    def filter_locations(self,locations: list, featuretype=None):
        """ locations whose raw['class'] is in classes or None

        Note than places will be returns by feature type order
        
        locs = filter_locations(locations,featuretype=['place', 'boundary'])  """
        if locations is None or featuretype is None:
            return locations
        r = [loc for ftype in featuretype for loc in locations if loc.raw['class'] == ftype]
        if len(r) == 0:
            return None
        else:
            return r

    def geocode_osm(self,place: str, featuretype: list, countries=None, search_all=False, distance_from=None):
        """geocode a place with open street map geocoder
        
        Args:
            place: string with place or address to geocode
            featuretype: type of feature (country, state, city, settlement)
            countries: list of countries to restrict the query in order.
                       can contain lists, e.g.:['pt','br',['ao','cv','mz'],['es','fr]]
            search_all: continue searching in more countries after first results
            distance_from: coordinates of origin. If more than one place located with same importance
                            return the closest one to this point. If None, ignore distance.



        See:
            https://geopy.readthedocs.io/en/stable/#nominatim
            https://geopy.readthedocs.io/en/stable/index.html?highlight=geopy.extra.rate_limiter.RateLimiter#geopy.extra.rate_limiter.RateLimiter
        """

        if countries is not None:
            if type(countries) is str:
                country_codes = [countries]
            elif type(countries) is list:
                if len(countries) > 0:
                    country_codes = countries
                else:
                    country_codes = None
            else:
                raise(ValueError("countries must be None, str or list"))
        else:
            country_codes = None
        
        if type(distance_from) is not None:
            if type(distance_from) is not tuple:
                raise(ValueError("distance_from must be a tuple (lat,long)"))
            elif len(distance_from) != 2:
                raise(ValueError("distance_from must be a tuple (lat,long)"))

        cache = []
        if featuretype is None:
            featuretype = ['boundary','place']

        locations = geocode(nplace, 
                        featuretype=featuretype,
                        exactly_one=False, 
                        namedetails=True, addressdetails=True,
                        country_codes=country_codes[0])

        locations = self.filter_locations(locations,featuretype)

        if (locations is None or search_all) and len(country_codes) > 1: 
                  
            for countries in country_codes[1:]:
                more_locations = geocode(nplace, 
                                featuretype=featuretype,
                                exactly_one=False,
                                namedetails=True, addressdetails=True,
                                country_codes=countries)
                more_locations = self.filter_locations(more_locations,featuretype)
                if more_locations is not None:
                    if locations is None:
                        locations = more_locations
                    else:
                        locations = locations + more_locations
                    if not search_all:
                        break 

        if locations is None:
            return None
        else:
            found = False
            for location in [loc for loc in locations]:
                    
                osm_id = location.raw.get('osm_id','*noif*')
                address_details = location.raw['address']
                osm_class = location.raw['class']
                osm_type = location.raw['type']
                lat = location.latitude
                long = location.longitude
                if distance_from is not None:
                    d_lat, d_long = distance_from
                    distance_in_km = geodesic((d_lat,d_long), (lat,long)).km
                else:
                    distance_in_km = None
                    
                country = address_details.get('country','')
                
                city = address_details.get('city',None)  # urban    
                town = address_details.get('town',None)  # rural
                municipality = address_details.get('municipality',None)  # rural

                city_name = city if city is not None \
                            else town if town is not None \
                            else municipality if municipality is not None \
                            else None
                # if city_name is None:
                #    continue
                address_details.pop('postcode',None)
                address_details.pop('country_code',None)

                address_details.pop('ISO3166-2-lvl4',None)
                address_details.pop('ISO3166-2-lvl3',None)

                osm_address = ", ".join(list(address_details.values()))  
                osm_importance = location.raw['importance']
                found = True
                cache = cache + [(place,'osm',osm_id, osm_address, city_name, country, osm_importance,osm_class,osm_type,lat,long,distance_in_km)]

            if found:        
                cache_df = pd.DataFrame(cache,columns=self.result_cols)
                # sort by descending importance and increasing distance. First row should be the best
                cache_df = cache_df.sort_values(['importance','distance'], ascending=[False,True]).iloc[:1]
                return cache_df  
            else:
                return None       

Prioridades de pesquisa

Pesquisa procede pela ordem definida de países. 

Isso permite dar prioridade a um lugar num país sobre ou com o mesmo nome noutro país.

Notar que é possível dar igual prioridade a um grupo de países, colocando-os numa lista. 
Nesse caso todos os lugares desses países são recolhidos e ordenados por importânica.

---

Search priorities

It is possible to define the search order and stop when there is a hit.

In [ ]:
pt = 'pt'
br = 'br'
plp = ['ao','mz','cv','st','mo','in']
other_countries = ['es','it','fr','ie','de']
all = None  # None in fact searches the whole planet
countries = [pt, br, plp,other_countries,all]

#### Normalização

---

#### Normalizing

TODO:
* Ler de ficheiro para Data Frame
* transformar Dataframe em dict
* adicionar entradas aqui
* no fim do processamento transformar de dict em dataframe.

In [252]:
import pandas as pd

places_normalization_file = '../inferences/places/places_normalization.csv'
nplaces_df = pd.read_csv(places_normalization_file)
nplaces_df = nplaces_df[nplaces_df.naturalidade != nplaces_df.normalizada]  # keep only diferents
nplaces = dict(nplaces_df.itertuples(index=False))

In [282]:
nplaces.update(
  {  # add "fauc_form":"normalized_form",  
"Alvações de Tanha":"Alvações do Tanha",
 "Cantarinha":"Cantarinhas",
 "Fontes Barrocas":"Fontes Barrosas", # -OSM, -geonames,+INE
 "Guizandaria":"Guizanderia", # zona do Carregado
 "Junqueiros":"Junqueiro",
 "Labrugeira":"Labrujeira",
 "Paredinha":"Paredinhas",
 "Salvaterra de Extremo":"Salvaterra do Extremo",
 "Salvaterra dos Magos":"Salvaterra de Magos",
 "Vila Chã da Graciosa":"Vila Chã da Braciosa",
 "Lontolim ou Loutolim, Salcete, Goa, Índia":"Loutolim, Salcete, Goa, Índia",
 "Lontulim":"Lontolim, Salcete, Goa, Índia",
 "Lontulim ou Loutolim": "Lontolim, Salcete, Goa, Índia"
  }
)


Remove normalized names from not found list, so they are searched again.

In [277]:
not_found_file = '../inferences/places/osm_not_found.csv'

not_found_df: pd.DataFrame = None

if exists(not_found_file):
    not_found_df = pd.read_csv(not_found_file)
else:
    not_found_df = pd.DataFrame(columns=['not_found'])

not_found_df = not_found_df[~not_found_df['not_found'].isin(nplaces.keys())]
not_found_df.to_csv(not_found_file,index=False)

#### Testes

---

#### Testing

Colocar o topónimo a localizar

---

Set the place to look for

In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.distance import geodesic

geolocator = Nominatim(user_agent="fauc1530-1919", timeout=2)
# abide to https://operations.osmfoundation.org/policies/nominatim/
geocode = RateLimiter(geolocator.geocode, min_delay_seconds = 1,   return_value_on_exception = None) 


In [278]:
place = "Loutolim"

if place in nplaces.keys():
    nplace = nplaces[place]
    print(f"Normalized: {nplace}")
else:
    nplace = place

uc_location = (40.207422, -8.4260033)


locator = GeocodeOSM('fauc1537-1919', timeout=15)
result = locator.geocode_osm(nplace, featuretype=['boundary','place'], countries=countries, distance_from=uc_location, search_all=False)
result

,place_name,geocoder,id,address,city,country,importance,class,type,latitude,longitude,distance
0,Loutolim,osm,4492463188,"Loutolim, Salcete, South Goa, Goa, India",None,India,0.308387,place,village,15.340017,73.986002,8289.154055


### Improve

* Try to remove "orago" from name when not found. Orago in São|Santa+word+da|de|do+Place

In [279]:
places = naturalidades.index
len(places)
places[:10]

Index(['Lisboa', 'Coimbra', 'Porto', 'Braga', 'Évora', 'Viseu', 'Guimarães',
       'Lamego', 'Aveiro', 'Vila Real'],
      dtype='object', name='value')

Nova versão com função

In [280]:
from os.path import exists
import pandas as pd

uc_location = (40.207422, -8.4260033)


locator = GeocodeOSM('fauc1537-1919', timeout=120)

cache_file = '../inferences/places/osm-places.csv'
places_cache_cols = ['naturalidade','geocoder','id','address','city','country','importance','class','type','latitude','longitude','distance']
if exists(cache_file):
    places_cache = pd.read_csv(cache_file, dtype={'id':'str'})
else:
   places_cache = pd.DataFrame(columns=locator.result_cols)


not_found_file = '../inferences/places/osm_not_found.csv'

not_found_df: pd.DataFrame = None

if exists(not_found_file):
    not_found_df = pd.read_csv(not_found_file)
    not_found = list(not_found_df['not_found'])
else:
    not_found = []
    not_found_df = pd.DataFrame(columns=['not_found'])

cached = set(places_cache['place_name'])

counter = 0
n_cached = 0
n_not_found = 0

for place in places:
    
    if len(place.strip()) == 0:
        continue

    if place in ['Alemanha','Lisboa','Aveiro','Ilha da Madeira']:
        pass  # to debug

    counter = counter + 1
    if place in cached:
        n_cached = n_cached + 1
        continue
    if place in not_found:
        n_not_found = n_not_found + 1
        continue

    if place in nplaces.keys():
        nplace = nplaces[place]
        print(f"Using normalized form '{nplace}' for '{place}'")
    else:
        nplace = place
    

    cache_df = locator.geocode_osm(nplace, featuretype=None, countries=countries, distance_from=uc_location, search_all=False)
    if cache_df is not None:
        cache_df['place_name'] = place  # restore the original name, before normalization
        places_cache = pd.concat([places_cache,cache_df],axis=0)
        print(f"Found: {place} ({cache_df['address'].iloc[0]})")
    else:
            nf = pd.DataFrame([(place)],columns=['not_found'])
            not_found_df = pd.concat([not_found_df,nf], axis=0) 
            not_found = not_found + [place]
            print("Not found:",place)






Using normalized form 'Cantarinhas' for 'Cantarinha'
Not found: Cantarinha
Using normalized form 'Fontes Barrosas' for 'Fontes Barrocas'
Not found: Fontes Barrocas
Using normalized form 'Guizanderia' for 'Guizandaria'
Not found: Guizandaria
Using normalized form 'Labrujeira' for 'Labrugeira'
Not found: Labrugeira
Using normalized form 'Loutolim, Salcete, Goa, Índia' for 'Lontolim ou Loutolim, Salcete, Goa, Índia'
Found: Lontolim ou Loutolim, Salcete, Goa, Índia (Loutolim, Salcete, South Goa, Goa, India)
Using normalized form 'Lontolim ou Loutolim, Salcete, Goa, Índia' for 'Lontulim'
Not found: Lontulim
Using normalized form 'Lontolim ou Loutolim, Salcete, Goa, Índia' for 'Lontulim ou Loutolim'
Not found: Lontulim ou Loutolim
Using normalized form 'Paredinhas' for 'Paredinha'
Not found: Paredinha
Using normalized form 'Vila Chã da Braciosa' for 'Vila Chã da Graciosa'
Not found: Vila Chã da Graciosa


In [257]:
places_cache.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6977 entries, 0 to 0
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   place_name  6977 non-null   object 
 1   geocoder    6977 non-null   object 
 2   id          6977 non-null   int64  
 3   address     6977 non-null   object 
 4   city        6754 non-null   object 
 5   country     6977 non-null   object 
 6   importance  6977 non-null   float64
 7   class       6977 non-null   object 
 8   type        6977 non-null   object 
 9   latitude    6977 non-null   float64
 10  longitude   6977 non-null   float64
 11  distance    6977 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 708.6+ KB


In [258]:
places_cache.to_csv(cache_file,index=False)



In [261]:
# transform normalized places into df
nplaces_df = pd.DataFrame([(p,n) for p,n in nplaces.items()],columns=['naturalidade','normalizada'])
nplaces_df.sort_values(['naturalidade','normalizada'], inplace=True)
nplaces_df.to_csv(places_normalization_file, index=False)

In [262]:
# Merge located places with naturalidade stats
naturalidades['naturalidade'] = naturalidades.index.values
natn = naturalidades.merge(nplaces_df,how='left',right_on='naturalidade',left_on='naturalidade')
not_normalized = natn[natn.normalizada.isnull()]['naturalidade']
natn.loc[natn.normalizada.isnull(),'normalizada'] = not_normalized


In [275]:
naturalidade_normalizada_file = "../inferences/places/naturalidade-normalizada.csv"
natn.to_csv(naturalidade_normalizada_file,index=False)

In [287]:

naturalidade_geocoded = natn.merge(places_cache,how='left',right_on='place_name',left_on='naturalidade')
naturalidade_geocoded.drop('place_name',axis=1, inplace=True)
naturalidade_geocoded.head(10)

,count,date_in,date_max,naturalidade,normalizada,geocoder,id,address,city,country,importance,class,type,latitude,longitude,distance
0,8784,1537-02-12,1916-07-19,Lisboa,Lisboa,osm,5400890.0,"Lisboa, Lisboa, Portugal",Lisboa,Portugal,0.824970,boundary,administrative,38.744052,-9.151828,174.057858
1,5526,1537-00-00,1915-10-12,Coimbra,Coimbra,osm,5379538.0,"Coimbra, Coimbra, Portugal",Coimbra,Portugal,0.685346,boundary,administrative,40.211193,-8.429463,0.511958
2,3391,1537-05-30,1917-10-22,Porto,Porto,osm,3372453.0,"Cedofeita, Santo Ildefonso, Sé, Miragaia, São ...",Porto,Portugal,0.735184,boundary,administrative,41.149451,-8.610788,105.770156
3,1608,1540-01-21,1914-07-24,Braga,Braga,osm,4115866.0,"Braga, Braga, Portugal",Braga,Portugal,0.671045,boundary,administrative,41.551058,-8.428005,149.213032
4,1072,1537-11-22,1910-10-10,Évora,Évora,osm,5402589.0,"Évora, Évora, Portugal",Évora,Portugal,0.633324,boundary,administrative,38.570774,-7.909281,187.077773
5,986,1537-00-00,1912-07-03,Viseu,Viseu,osm,5330332.0,"Viseu, Viseu, Portugal",Viseu,Portugal,0.621902,boundary,administrative,40.657471,-7.913866,66.226034
6,980,1537-12-18,1912-07-18,Guimarães,Guimarães,osm,3924938.0,"Guimarães, Braga, Portugal",Guimarães,Portugal,0.661330,boundary,administrative,41.441768,-8.295571,137.515578
7,972,1537-00-00,1909-10-05,Lamego,Lamego,osm,5328700.0,"Lamego, Viseu, Portugal",Lamego,Portugal,0.578091,boundary,administrative,41.071740,-7.814995,109.011191
8,790,1538-04-21,1913-10-13,Aveiro,Aveiro,osm,5325138.0,"Aveiro, Aveiro, Portugal",Aveiro,Portugal,0.625496,boundary,administrative,40.640496,-8.653784,51.829301
9,765,1537-03-07,1909-11-09,Vila Real,Vila Real,osm,4202166.0,"Vila Real, Vila Real, Portugal",Vila Real,Portugal,0.678191,boundary,administrative,41.300624,-7.764228,133.643484


In [288]:
# save naturalidade geocoding status

naturalidade_status_file = "../inferences/places/naturalidade_geo_status.csv"
naturalidade_geocoded.to_csv(naturalidade_status_file, index=False)


## Número de estudantes com naturalidade georeferenciada

---

## Number of students with geocoded place of birth

In [264]:
naturalidade_geocoded[naturalidade_geocoded['address'].notnull()][['count']].sum()

count    88366
dtype: int64

### Lugares georeferenciados com mais estudantes

---

### georeferenced places with more students

In [266]:
naturalidade_geocoded.groupby(['normalizada'])['count'].sum().sort_values(ascending = False).head(50)

normalizada
Lisboa                    8784
Coimbra                   5526
Porto                     3391
Braga                     1608
Rio de Janeiro, Brasil    1126
Évora                     1072
Viseu                      986
Guimarães                  980
Lamego                     972
Salvador, Brasil           906
Aveiro                     790
Vila Real                  765
Santarém                   728
Viana do Castelo           706
Leiria                     668
Ilha da Madeira            569
Portalegre                 542
Castelo Branco             538
Tomar                      531
Barcelos                   527
Beja                       500
Guarda                     475
Elvas                      448
Abrantes                   430
Montemor-o-Velho           426
Setúbal                    425
Ponte de Lima              397
Covilhã                    393
Bragança                   376
Torres Novas               360
Soure                      359
Faro                       

Exemplo

In [286]:
naturalidade_geocoded[naturalidade_geocoded['naturalidade'].isin(['Casais do Campo'])]

,count,date_in,date_max,naturalidade,normalizada,geocoder,id,address,city,country,importance,class,type,latitude,longitude,distance
549,20,1697-10-01,1849-10-23,Casais do Campo,"Casais, S. Martinho do Bispo, Coimbra",osm,251633008.0,"Casais, Coalhadas, São Martinho do Bispo e Rib...",Coimbra,Portugal,0.87,place,neighbourhood,40.202924,-8.483113,4.887821


Sem georeferenciação

---

Without geocoding

In [268]:
naturalidade_geocoded[naturalidade_geocoded['address'].isnull()][['count']].sum()

count    5779
dtype: int64

In [269]:
not_found = naturalidade_geocoded[naturalidade_geocoded['address'].isnull()]['naturalidade'].values
not_found_df = pd.DataFrame(not_found,columns=['not_found'])
not_found_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4548 entries, 0 to 4547
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   not_found  4548 non-null   object
dtypes: object(1)
memory usage: 35.7+ KB


In [270]:
not_found_df.head()

,not_found
0,
1,Ribeira de Soares
2,Treixomil
3,Ribeira de Homem
4,São Martinho de Lagares


In [271]:

not_found_file = '../inferences/places/osm_not_found.csv'
not_found_df.to_csv(not_found_file,index=False)

Lugares mais importantes para georeferenciar

---

Most relevant places to be geocoded

In [272]:
naturalidade_geocoded[naturalidade_geocoded['address'].isnull()].sort_values('count',ascending=False).head(20)

,count,date_in,date_max,naturalidade,normalizada,place_name,geocoder,id,address,city,country,importance,class,type,latitude,longitude,distance
387,32,1540-00-00,1771-05-24,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
872,11,1586-10-10,1745-10-01,Treixomil,Treixomil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
862,11,1705-10-01,1766-10-01,Ribeira de Soares,Ribeira de Soares,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989,9,1678-10-01,1767-10-01,Ribeira de Homem,Ribeira de Homem,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,9,1677-10-01,1886-10-15,São Martinho de Lagares,São Martinho de Lagares,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1004,9,1674-10-01,1766-10-01,Traveira,Traveira,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1089,8,1639-10-19,1736-11-19,Urzelhe,Urzelhe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1091,8,1683-10-01,1909-10-07,Vale de Ladrões,Vale de Ladrões,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1073,8,1665-10-21,1768-10-01,Salir do Mato,Salir do Mato,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1027,8,1563-10-01,1748-10-01,Cazegas,Cazegas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
